In [1]:
import os

import pandas as pd
import numpy as np

In [2]:
ROOT_PATH = '/'.join(os.getcwd().split('/')[:-4])
DATA_PATH = f'{ROOT_PATH}/Dataset/books/'

In [3]:
books   = pd.read_csv(f'{DATA_PATH}/Books.csv')
ratings = pd.read_csv(f'{DATA_PATH}/Ratings.csv') 

/tmp/ipykernel_2341624/3734577718.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books   = pd.read_csv(f'{DATA_PATH}/Books.csv')


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
df = books.merge(ratings, how = 'left', on = 'ISBN')
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676.0,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385.0,0.0


In [7]:
df.isnull().sum()

ISBN                      0
Book-Title                0
Book-Author               1
Year-Of-Publication       0
Publisher                 2
Image-URL-S               0
Image-URL-M               0
Image-URL-L               4
User-ID                1209
Book-Rating            1209
dtype: int64

In [8]:
df = df.dropna(subset = ['Book-Rating', 'User-ID', 'Image-URL-L', 
                         'Publisher'  , 'Book-Author'])

## 평점이 1점 이상인 녀석들만 가져옴.
df = df[df['Book-Rating'] > 0]
df['Book-Rating'].value_counts()

8.0     91803
10.0    71225
7.0     66401
9.0     60776
5.0     45355
6.0     31687
4.0      7617
3.0      5118
2.0      2375
1.0      1481
Name: Book-Rating, dtype: int64

In [9]:
## 특정 칼럼이 갯수가 n개 미만인 데이터를 제거하는 함수
def eliminate_data(df, column, n):
    
    ## column을 통한 평점 갯수 산출
    rating_counts = pd.DataFrame(df[column].value_counts())
    
    ## 평점 데이터가 n개 이상인 데이터의 인덱스 값만 추출
    rare_idx      = rating_counts[rating_counts[column] >= n].index
    
    ## 전체 데이터에서 평점 데이터가 n개 미만인 데이터 제거
    return df[df[column].isin(rare_idx)]

In [10]:
common_books = eliminate_data(df, 'Book-Title', 100)
common_books['Book-Title'].value_counts()

The Lovely Bones: A Novel     707
Wild Animus                   581
The Da Vinci Code             494
The Secret Life of Bees       406
The Nanny Diaries: A Novel    393
                             ... 
Congo                         100
Deception Point               100
Daddy's Little Girl           100
Cause of Death                100
Disclosure                    100
Name: Book-Title, Length: 200, dtype: int64

In [11]:
## 평가를 한 갯수가 10개 이상인 유저들만 추출
common_books = eliminate_data(common_books, 'User-ID', 10)
common_books['User-ID'].value_counts()

11676.0     225
16795.0      81
95359.0      61
60244.0      46
240567.0     42
           ... 
232106.0     10
104113.0     10
78834.0      10
74286.0      10
208671.0     10
Name: User-ID, Length: 434, dtype: int64

In [12]:
## 유저 - 책 DataFrame 생성
user_book_df = common_books.pivot_table(index  = ['User-ID'], columns = ['Book-Title'],
                                        values = 'Book-Rating')

## 랜덤하게 데이터 프레임에서 유저 추출
random_user    = int(pd.Series(user_book_df.index).sample(1).values)
random_user_df = user_book_df[user_book_df.index == random_user]
random_user_df

Book-Title,1984,1st to Die: A Novel,2nd Chance,A Bend in the Road,"A Child Called \It\"": One Child's Courage to Survive""",A Heartbreaking Work of Staggering Genius,A Is for Alibi (Kinsey Millhone Mysteries (Paperback)),A Map of the World,A Painted House,A Prayer for Owen Meany,...,Watership Down,We Were the Mulvaneys,When the Wind Blows,Where the Heart Is (Oprah's Book Club (Paperback)),While I Was Gone,White Oleander : A Novel,White Oleander : A Novel (Oprah's Book Club),Wicked: The Life and Times of the Wicked Witch of the West,Wild Animus,"\O\"" Is for Outlaw"""
User-ID,,,,,,,,,,,,,,,,,,,,,
101620.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
books_read = random_user_df.columns[random_user_df.notna().any()].tolist()
books_read, len(books_read)

(['A Painted House',
  'A Time to Kill',
  "Bridget Jones's Diary",
  'Bridget Jones: The Edge of Reason',
  'Four To Score (A Stephanie Plum Novel)',
  'Hard Eight : A Stephanie Plum Novel (A Stephanie Plum Novel)',
  'One for the Money (Stephanie Plum Novels (Paperback))',
  'Seven Up (A Stephanie Plum Novel)',
  'Skipping Christmas',
  'The Brethren',
  'The Partner',
  'The Summons',
  'Two for the Dough'],
 13)

In [14]:
## 랜덤한 유저를 통해 같은 책을 읽은 사람을 df로 생성
books_read_df = user_book_df[books_read]
books_read_df

Book-Title,A Painted House,A Time to Kill,Bridget Jones's Diary,Bridget Jones: The Edge of Reason,Four To Score (A Stephanie Plum Novel),Hard Eight : A Stephanie Plum Novel (A Stephanie Plum Novel),One for the Money (Stephanie Plum Novels (Paperback)),Seven Up (A Stephanie Plum Novel),Skipping Christmas,The Brethren,The Partner,The Summons,Two for the Dough
User-ID,,,,,,,,,,,,,
254.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638.0,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
2766.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN
4017.0,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
274004.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
274301.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
276050.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
## 유저별로 같은 책을 읽은 권수 집계
user_book_count = books_read_df.T.notnull().sum()
user_book_count

User-ID
254.0       0
638.0       2
2766.0      1
4017.0      1
6242.0      0
           ..
274004.0    0
274301.0    0
276050.0    0
276165.0    0
276231.0    0
Length: 434, dtype: int64

In [16]:
user_book_count = user_book_count.reset_index()
user_book_count.columns = ['User_id', 'book_count']
user_book_count.sort_values('book_count', ascending = False)

,User_id,book_count
154,101620.0,13
19,11676.0,12
161,104636.0,9
25,16795.0,7
399,258534.0,6
...,...,...
214,138844.0,0
215,138995.0,0
216,139913.0,0
223,143163.0,0


In [17]:
## 랜덤하게 추출한 유저와 같은 책을 읽은 유저가 5권 이상인 유저들만 추출
users_same_book = user_book_count[user_book_count['book_count'] >= 5]['User_id']
users_same_book

8        6575.0
10       7346.0
19      11676.0
25      16795.0
72      43246.0
90      60244.0
154    101620.0
161    104636.0
174    110934.0
179    114368.0
193    125519.0
208    135149.0
249    162639.0
254    165308.0
321    217318.0
371    240567.0
372    241980.0
380    245410.0
399    258534.0
Name: User_id, dtype: float64

In [18]:
final_df = pd.concat([books_read_df[books_read_df.index.isin(users_same_book)],
                      random_user_df[books_read]])

## 상관관계 계산 및 데이터 정렬
corr_df   = final_df.T.corr().unstack().sort_values().drop_duplicates()
corr_df   = pd.DataFrame(corr_df, columns = ['corr'])
corr_df.index.names = ['user_id1', 'user_id2']

## 계산된 상관관계 df에서 user_id1이 랜덤 유저와 동일한 데이터만 추출
corr_df   = corr_df.reset_index()
top_users = corr_df[(corr_df['user_id1'] == random_user)][['user_id2', 'corr']].reset_index(drop = True)

top_users

,user_id2,corr
0,110934.0,-0.790569
1,245410.0,-0.612372
2,165308.0,-0.326637
3,43246.0,0.307470
4,135149.0,0.456435
5,135149.0,0.456435
6,258534.0,0.500000
7,60244.0,0.745356
8,114368.0,0.780821
9,104636.0,0.858696


In [19]:
## 계산된 상관관계 df에서 user_id1과 랜덤 유저가 같으면서, 음의 상관관계가 0.6 이상인 데이터만 추출
top_users = corr_df[(corr_df['user_id1'] == random_user) & (corr_df['corr'] > 0.6)]
top_users = top_users[['user_id2', 'corr']].reset_index(drop = True)
top_users = top_users.sort_values(by = 'corr', ascending = False)
top_users.rename(columns = {'user_id2' : 'User-ID'}, inplace = True)
top_users

,User-ID,corr
7,101620.0,1.000000
6,6575.0,0.886659
5,241980.0,0.875000
4,241980.0,0.875000
3,240567.0,0.861858
2,104636.0,0.858696
1,114368.0,0.780821
0,60244.0,0.745356


In [20]:
top_users_ratings = top_users.merge(ratings[['User-ID', 'ISBN', 'Book-Rating']], how = 'inner')
top_users_ratings = top_users_ratings[top_users_ratings['User-ID'] != random_user]

top_users_ratings

,User-ID,corr,ISBN,Book-Rating
50,6575.0,0.886659,0001714600,0
51,6575.0,0.886659,0028604199,0
52,6575.0,0.886659,0028606736,0
53,6575.0,0.886659,0030640075,0
54,6575.0,0.886659,0060002093,0
...,...,...,...,...
5484,60244.0,0.745356,B00005LD7A,0
5485,60244.0,0.745356,B00005TZWI,9
5486,60244.0,0.745356,B000068RVK,0
5487,60244.0,0.745356,B00016560C,0


In [21]:
## weighted score 계산 (유저와의 상관관계 * 레이팅 점수)
top_users_ratings['weighted_rating'] = top_users_ratings['corr'] * top_users_ratings['Book-Rating']
top_users_ratings

,User-ID,corr,ISBN,Book-Rating,weighted_rating
50,6575.0,0.886659,0001714600,0,0.000000
51,6575.0,0.886659,0028604199,0,0.000000
52,6575.0,0.886659,0028606736,0,0.000000
53,6575.0,0.886659,0030640075,0,0.000000
54,6575.0,0.886659,0060002093,0,0.000000
...,...,...,...,...,...
5484,60244.0,0.745356,B00005LD7A,0,0.000000
5485,60244.0,0.745356,B00005TZWI,9,6.708204
5486,60244.0,0.745356,B000068RVK,0,0.000000
5487,60244.0,0.745356,B00016560C,0,0.000000


In [22]:
recommendation_df = top_users_ratings.groupby('ISBN').agg({'weighted_rating' : 'mean'})
recommendation_df = recommendation_df.reset_index()
recommendation_df

,ISBN,weighted_rating
0,0001714600,0.000000
1,0006345549,0.000000
2,0007103468,0.000000
3,0007154615,5.962848
4,0020125305,0.000000
...,...,...
4596,B00005TZWI,6.708204
4597,B00005Y0HR,8.866586
4598,B000068RVK,0.000000
4599,B00016560C,0.000000


In [23]:
books_recommend = recommendation_df[recommendation_df['weighted_rating'] > 6.5]
books_recommend = books_recommend.sort_values('weighted_rating', ascending = False).head(10)
books_recommend.merge(df[['ISBN', 'Book-Title']]).drop_duplicates()

,ISBN,weighted_rating,Book-Title
0,0843733756,8.866586,European Detours: A Travel Guide to Unusual Si...
1,015602943X,8.866586,The Time Traveler's Wife (Harvest Book)
27,0836218310,8.866586,Far Side Gallery 3 Paperback
43,0671729489,8.750000,Seeds of Yesterday (Dollanganger)
55,0446690791,8.750000,"The Odd Todd Handbook: Hard Times, Soft Couch"
56,0743216490,8.750000,Hollywood Divorces (Ay Adult - Collins)
62,0743216342,8.750000,Hollywood Wives : The New Generation
65,0743206029,8.750000,On the Street Where You Live : A Novel
109,0446356603,8.750000,Lovers &amp; Gamblers
